In [1]:
##Code Section for Declaring Imports##

from bs4 import BeautifulSoup # Load the libraries for beautiful soup
import requests
#import urllib2
import time 
import pandas as pd

# 1. Code Section for Beautiful Soup Headers

In [55]:
##Code Section for Beautiful Soup Headers##

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://fbref.com/en/comps/9/1526/2016-2017-Premier-League-Stats"
pageTree = requests.get(page, headers=headers)
pageSoup_teams = BeautifulSoup(pageTree.content, 'html.parser')

# 2. Code Section to fetch the Teams - Source : Fbref.com

In [3]:
##Code Section to fetch the Teams ##

Player_Teams = []
table = pageSoup_teams.find('table', class_ = 'min_width sortable stats_table')
for link in table.find_all('a'):
        temp = "https://fbref.com" + link.get("href")
        team_name = link.text
        Player_Teams.append([team_name,temp])
Player_Teams_upd = []
for value in Player_Teams:
    if value[1][-1:-6:-1] == 'statS':
        Player_Teams_upd.append(value)
        
Player_Teams_upd;        

### 2.1 Code Section to fetch the Players from Teams - Source : Fbref.com

In [6]:
##Code Section to fetch Player List from Teams ##

Players_Info = {}
for player_page in Player_Teams_upd:
    page = player_page[1]
    player_team = player_page[0]
    pageTree = requests.get(page, headers=headers)
    pageSoup_teams = BeautifulSoup(pageTree.content, 'html.parser')
    
    for table in pageSoup_teams.find_all('div', id = 'div_stats_standard_1526'):
        rows = [row for row in table.find_all('tr')]
    for index in range(2,len(rows)-2):
        player_name =     rows[index].find_all('th')[0].find('a').text
        player_url  =     'https://fbref.com' + rows[index].find_all('th')[0].find('a')['href']
        player_details =  rows[index].find_all('td')
        Players_Info[player_name] = [player_name] + [player_url] + [player_team] + [i.text for i in player_details[:-1]] 

Players_Info_df = pd.DataFrame(Players_Info).T  #transpose
Players_Info_df.columns = ['Player','PlLink','Team','Nation', 'Pos','Age','MP','Starts','Min','90s','Gls','Ast', 
                    'PK','PKatt','CrdY','CrdR','PGls','PAst','PG+A','PG-PK','PG+A-PK'] 
Players_Info_df.set_index('Player', inplace= True)
Players_Info_df;        

### 2.2 Code Section to fetch Player level Attributes - Source : Fbref.com

In [52]:
##Code Section to fetch Specific Player Attributes##
Player_Attr = {}
Player_Shooting = {}
for index, row in enumerate(Players_Info_df.head(2).itertuples()):
    page = row.PlLink
    pageTree = requests.get(page, headers=headers)
    pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')

    #pageSoup_player 
    for table in pageSoup_player.find_all('div', class_ = 'players', id = 'info'):
        #print(table)
        rows = [row for row in table.find_all('p')]
        headings = [heading for heading in table.find_all('h1') ]
    try:   
        #print(headings[0])
        player_name   = headings[0].find('span').text
        #print(player_name)
        player_height = rows[2].find_all('span')[0].text
        player_weight = rows[2].find_all('span')[1].text
        player_footedness = rows[1].text
        Player_Attr[player_name] = [player_name] + [player_height] + [player_weight] + [player_footedness]
    except:
        player_name   = 'Error'
        player_height = 'Error'
        player_weight = 'Error'
        player_footedness = 'Error'
        Player_Attr[player_name] = [player_name] + [player_height] + [player_weight] + [player_footedness]
        continue
#    for table in pageSoup_player.find_all('div', id = 'div_stats_shooting_dom_lg'):
#        rows = [row for row in table.find_all('tr')]
        #for index in range(4:18): 
#        items = rows[12].find_all('td')
#        Player_Shooting[player_name] = [player_name] + [i.text for i in items[4:]]

#print(Player_Shooting)
#Players_Shooting_df = pd.DataFrame(Player_Shooting).T 
#Players_Shooting_df.columns = ['Player','','Weight','Footedness']#print(rows[3])

Players_Attr_df = pd.DataFrame(Player_Attr).T  #transpose
Players_Attr_df.columns = ['Player','Height','Weight','Footedness'] 
Players_Attr_df.set_index('Player', inplace= True)
## Merge the above different Player specific information into a single DataFrame for players
Players_Info_outer_merged = pd.merge(Players_Info_df, Players_Attr_df,
                        how="outer", on=["Player"])

# 3. Code Section to fetch markvet value of the Players - Source : Transfermarkt.com

### 3.1 Beautiful Soup Header for Transfermark.com website(for market valuations)

In [57]:
##Code Section to fetch Player Market Valuations from TransferMarket Website##
page = "https://www.transfermarkt.us/premier-league/daten/wettbewerb/GB1"
pageTree = requests.get(page, headers=headers)
pageSoup_teams = BeautifulSoup(pageTree.content, 'html.parser')

### 3.2 Fetch the teams

In [62]:
Teams = []
Teams_mod = []
for table in pageSoup_teams.find_all('table', class_='items'):
    #time.sleep(20)
    for link in table.find_all('a'):
        temp = "https://www.transfermarkt.us" + link.get("href")
        Teams.append(temp)
  ## Notice that there are redundant links in the list so I am filtering out the duplicate values      
    [Teams_mod.append(x) for x in Teams if x not in Teams_mod]

Teams_corr = Teams_mod[4::2] #This is to clean-up redundant and un-necessary links in the data     
Teams_corr;

### 3.3 Fetch players from each of the teams

In [79]:
Teams_Players_Names = []
Teams_Players_Valuations = {}
Teams_Players_Positions = []
Teams_Players = []
Teams_Players_mod = []
team_page_temp = []
player_valuations_dataframe = pd.DataFrame()
#for team_pa'https://www.transfermarkt.us/chelsea-fc/startseite/verein/631?saison_id=20ge in Teams_corr: ## For each team in the list -> we go ahead to fetch the players
test_page = "https://www.transfermarkt.us/chelsea-fc/startseite/verein/631?saison_id=2016"
#team_page_temp.append(Teams_corr[0:3])
team_page_temp.append(test_page)
for team_page in team_page_temp: ## We Look through the list and fetch every team and load their respective pages 
    #time.sleep(20)
    #print(team_page)
    pageTree = requests.get(team_page, headers=headers)
    pageSoup_player = BeautifulSoup(pageTree.content, 'html.parser')
    for table in pageSoup_player.find_all('table', class_='items'):
        for link in table.find_all('a', class_ = 'spielprofil_tooltip'):
            if (link.get("title")) not in Teams_Players_Names:
                Teams_Players_Names.append(link.get("title"))
                #Player_Attr[link.get("title")] = link.get("title")
        for link in table.find_all('td', class_ = 'rechts hauptlink'):
                Teams_Players_Valuations.append(link.text)
                
Players_Attr_df = pd.DataFrame()  #transpose
#Players_Attr_df.columns = ['Player','Valuation'] 
Players_Attr_df['Player'] = Teams_Players_Names
Players_Attr_df['Valuation'] = Teams_Players_Valuations
Players_Attr_df.set_index('Player', inplace= True)

In [99]:
Players_Attr_df;

### 3.3 Fetch information from Wikipedia

In [104]:
import pageviewapi
page_views = pageviewapi.per_article('en.wikipedia', 'Rooney', '20151106', '20151120',
                        access='all-access', agent='all-agents', granularity='daily')
#print(page_views['items'])

### 3.4 Merge with the Final Dataset

In [88]:

Players_Dataset = pd.merge(Players_Attr_df,Players_Info_outer_merged,
                        how="outer", on=["Player"])

In [97]:
Players_Dataset.groupby("Team").head(25);

# 4. Data Models 